In [ ]:
from __future__ import division

In [ ]:
def rho_water(T=298.15, T0=None, units=None, a=None, just_return_a=False):
    """
    Density of water (kg/m3) as function of temperature (K)
    according to VSMOW model between 0 and 40 degree Celsius.
    Fitted using Thiesen's equation.
    
    Parameters
    ----------
    T: float
        Temperature (default: in Kelvin)
    T0: float
        Value of T for freezing point of water (default: 273.15)
    units: object (optional)
        object with attributes: Kelvin, meter, kilogram
    a: array_like (optional)
        5 parameters to the equation.
    just_return_a: bool (optional, default: False)
        Do not compute rho, just return the parameters ``a``.
        
    Returns
    -------
    Density of water (float if T is float and units is None)
    
    References
    ----------
    TANAKA M., GIRARD G., DAVIS R., PEUTO A. and BIGNELL N.,
        "Recommanded table for the density of water between 0 °C and 40 °C based on recent experimental reports",
        Metrologia, 2001, 38, 301-309. doi:10.1088/0026-1394/38/4/3
    """
    if units is None:
        K = 1
        m = 1
        kg = 1
    else:
        K = units.Kelvin
        m = units.meter
        kg = units.kilogram
    m3 = m**3
    if a is None:
        a = (-3.983035*K,  # C
            301.797*K, # C
            522528.9*K*K, # C**2
            69.34881*K, # C
            999.974950*kg/m3) # kg / m**3
    if just_return_a:
        return a
    if T0 is None:
        T0 = 273.15*K # K
    t = T - T0
    return a[4]*(1-((t + a[0])**2*(t + a[1]))/(a[2]*(t + a[3])))

In [ ]:
# No dependencies outside standard library, sane defaults.
rho_water()

In [ ]:
# It plays nice with SymPy
import sympy
a = sympy.symbols('a:5')
T, T0 = sympy.symbols('T T0')
rho_water(T, a=a, T0=T0)

In [ ]:
# ...which is useful if you want the equation in another language than Python
print("C")
print(sympy.ccode(rho_water(T, a=rho_water(just_return_a=True), T0=T0)))
print("\nFortran")
print(sympy.fcode(rho_water(T, a=rho_water(just_return_a=True), T0=T0), source_format='free'))

In [ ]:
# It plays nice with quantities
import quantities as pq
rho_water(298.15*pq.K, units=pq), rho_water(just_return_a=True, units=pq)

In [ ]:
# It plays nice with NumPy
import matplotlib.pyplot as plt
import numpy as np
C_plot = np.linspace(0, 40)
plt.plot(C_plot, rho_water(C_plot+273.15))
plt.xlabel('Temperature / $^\circ C$')
plt.ylabel('Density / $kg m^{-3}$')
_ = plt.title("Density of water vs. temperature")

In [ ]:
from math import log
import quantities as pq

In [ ]:
def eps_water(T=298.15, P=1.0):
    """
    Permitivity of water according to the parametrization by Bradley and Pitzer
    (empirical parametrization)
    
    Parameters
    ----------
    T: float
        Temperature in Kelvin
    P: float
        Pressure in bar
        
    References
    ----------
    - Bradley, D.J.; Pitzer, K.S. `Thermodynamics of electrolytes. 12. Dielectric
        properties of water and Debye--Hueckel parameters to 350/sup 0/C and 1 kbar`,
        J. Phys. Chem.; Journal Volume 83 (12) (1979), pp. 1599-1603
        http://pubs.acs.org/doi/abs/10.1021/j100475a009
        DOI: 10.1021/j100475a009
    """
    from math import exp
    from math import log as ln
    U = [0,  3.4279e2, -5.0866e-3, 9.4690e-7, -2.0525, 3.1159e3, 
        -1.8289e2, -8.0325e3, 4.2142e6, 2.1417]
    B  = U[7] + U[8]/T + U[9]*T
    C  = U[4] + U[5]/(U[6]+T)
    eps1000 = U[1]*exp(U[2]*T+U[3]*T**2)
    return eps1000 + C*ln((B+P)/(B+1000.0))

In [ ]:
def debye_huckel_A(eps_r, T, rho, b0, constants=None, one=1):
    """
    from Atkins
    
    Parameters
    ----------
    eps_r: float
        relative permittivity
    T: float with unit
        temperature
    rho: float with unit
        density
    b0: float with unit
        reference molality
    constants: object (optional)
        
    """
    if constants is None:
        import quantities
        constants = quantities.constants
    F = constants.Faraday_constant
    NA = constants.Avogadro_constant
    eps0 = constants.vacuum_permittivity
    kB = constants.Boltzmann_constant
    pi = constants.pi
    A = F**3/(4*pi*NA)*(rho*b0/(2*(eps0*eps_r*kB*NA*T)**3))**(one/2)
    return A

In [ ]:
from sympy import init_printing; init_printing()
from sympy import Symbol, symbols, S
class Dummy:
    pass
consts = Dummy()
consts.Faraday_constant = Symbol('F')
consts.Avogadro_constant = Symbol('N_A')
consts.vacuum_permittivity = Symbol('\epsilon_0')
consts.Boltzmann_constant = Symbol('k_B')
consts.pi = Symbol('pi')
debye_huckel_A(*symbols('epsilon_r T rho b^0'), constants=consts, one=S(1))

In [ ]:
debye_huckel_A(1, 1, 1, 1, constants=pq.constants).simplified

In [ ]:
T = 297 # 24 deg C
res = debye_huckel_A(eps_water(T), T*pq.K, rho_water(T)*pq.kg/pq.metre**3, 1*pq.mole/pq.kg)
res

In [ ]:
res.simplified

In [ ]:
res.simplified/log(10)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
C_plot = np.linspace(0, 40)
plt.plot(C_plot, np.array([
    debye_huckel_A(eps_water(T), T*pq.K, rho_water(T)*pq.kg/pq.metre**3,
                   1*pq.mole/pq.kg).simplified for T in 273.15+C_plot])/log(10))
plt.xlabel('Temperature / degC')
plt.ylabel('A')
_ = plt.title('Temperature dependence of the Debye-Huckel constant A')

In [ ]:
def debye_huckel_B(eps_r, T, rho, b0, constants=None, one=1):
    """
    from Atkins
    
    Parameters
    ----------
    eps_r: float
        relative permittivity
    T: float with unit
        temperature
    rho: float with unit
        density
    b0: float with unit
        reference molality
    constants: object (optional)
        attributes accessed: molar_gas_constant, Faraday_constant
    """
    if constants is None:
        import quantities
        constants = quantities.constants
    F = constants.Faraday_constant
    eps0 = constants.vacuum_permittivity
    R = constants.molar_gas_constant
    B = F*(2*rho*b0/(eps_r*eps0*R*T))**(one/2)
    return B

In [ ]:
consts.molar_gas_constant = Symbol('R')
debye_huckel_B(*symbols('epsilon_r T rho b0'), constants=consts, one=S(1))

In [ ]:
debye_huckel_B(1, 1, 1, 1, constants=pq.constants).simplified

In [ ]:
T = 297 # 24 deg C
res = debye_huckel_B(eps_water(T), T*pq.K, rho_water(T)*pq.kg/pq.metre**3, 1*pq.mol/pq.kg)
res.rescale(1/pq.angstrom)

In [ ]:
float(res.rescale(1/pq.angstrom))*T**0.5*eps_water(T)**0.5

In [ ]:
x, y = C_plot, np.array([
    debye_huckel_B(eps_water(T), T*pq.K, rho_water(T)*pq.kg/pq.metre**3,
                   1*pq.mole/pq.kg).rescale(1/pq.nm) for T in 273.15+C_plot])
plt.plot(x, y)
p = np.polyfit(x, y, 1)
fmtstr = r'$B / nm^{{-1}} = {0:.2f} + T \cdot {1:.2f} nm^{{-1}} ^{{\circ}}C^{{-1}}$'
fmtstr = r'$B / nm^-1 = {0:.2f} + T \cdot {1:.2f} nm^-1 ^\circC^-1$'
fmtstr = r'$B / nm^{{-1}} = {0:.2f} + {1:.4f} \cdot T /(^{{\circ}}C)^{{-1}}$'
plt.plot(x, np.polyval(p, x), '--', label=fmtstr.format(p[1], p[0]))
plt.xlabel(r'$T$ / $^{\circ}C$')
plt.ylabel('$B / nm^{-1}$')
plt.legend(loc='best')
_ = plt.title('Temperature dependence of the Debye-Huckel constant B')